In [ ]:
import os
import cv2
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
#unzipping rar file

!unrar x train.rar



UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from train.rar

Creating    dataset                                                   OK
Creating    dataset/Abhir                                             OK
Extracting  dataset/Abhir/Abir.6.3051.1.jpg                                0%  OK 
Extracting  dataset/Abhir/Abir.6.3051.10.jpg                               0%  OK 
Extracting  dataset/Abhir/Abir.6.3051.100.jpg                              0%  OK 
Extracting  dataset/Abhir/Abir.6.3051.101.jpg                              0%  OK 
Extracting  dataset/Abhir/Abir.6.3051.11.jpg                               0%  OK 
Extracting  dataset/Abhir/Abir.6.3051.12.jpg                               0%  OK 
Extracting  dataset/Abhir/Abir.6.3051.13.jpg                               0%  OK 
Extracting  dataset/Abhir/Abir.6.3051.14.jpg                               0%  OK 
Extracting  dat

In [ ]:
dataset_path = '/content/dataset'

In [ ]:
# Preprocessing the images
def load_images(dataset_path):
    images = []
    labels = []
    for person_name in os.listdir(dataset_path):
        person_folder = os.path.join(dataset_path, person_name)
        for image_name in os.listdir(person_folder):
            image_path = os.path.join(person_folder, image_name)
            image = cv2.imread(image_path)
            image = cv2.resize(image, (224, 224))  # Resizing images to 224x224
            images.append(image)
            labels.append(person_name)
    return np.array(images), np.array(labels)


images, labels = load_images(dataset_path)
images = images / 255.0  # Normalizing pixel values

In [ ]:
# Encoding labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)
labels = to_categorical(labels)

In [ ]:
# Splitting the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

In [ ]:
# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

datagen.fit(X_train)

In [ ]:
#Building Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

def create_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.25),

        Conv2D(64, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.25),

        Conv2D(128, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.25),

        Flatten(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),

        Dense(10, activation='softmax')  # 10 classes for 10 people
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = create_model()
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 batch_normalization (Batch  (None, 222, 222, 32)      128       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 111, 111, 32)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 batch_normalization_1 (Bat  (None, 109, 109, 64)      2

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

#  early stopping and learning rate reduction
early_stopping = EarlyStopping(patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(factor=0.5, patience=5)

history = model.fit(datagen.flow(X_train, y_train, batch_size=32),
                    validation_data=(X_val, y_val),
                    epochs=100,
                    callbacks=[early_stopping, reduce_lr])


Epoch 1/100
26/26 [==============================] - 166s 6s/step - loss: 1.1886 - accuracy: 0.6356 - val_loss: 11.3647 - val_accuracy: 0.1244 - lr: 0.0010
Epoch 2/100
26/26 [==============================] - 163s 6s/step - loss: 0.7444 - accuracy: 0.7500 - val_loss: 16.0327 - val_accuracy: 0.1194 - lr: 0.0010
Epoch 3/100
26/26 [==============================] - 162s 6s/step - loss: 0.5006 - accuracy: 0.8259 - val_loss: 16.6428 - val_accuracy: 0.1194 - lr: 0.0010
Epoch 4/100
26/26 [==============================] - 169s 6s/step - loss: 0.4311 - accuracy: 0.8557 - val_loss: 20.7188 - val_accuracy: 0.1194 - lr: 0.0010
Epoch 5/100
26/26 [==============================] - 160s 6s/step - loss: 0.4177 - accuracy: 0.8595 - val_loss: 20.2015 - val_accuracy: 0.1194 - lr: 0.0010
Epoch 6/100
26/26 [==============================] - 163s 6s/step - loss: 0.3708 - accuracy: 0.8644 - val_loss: 21.6015 - val_accuracy: 0.1194 - lr: 0.0010
Epoch 7/100
26/26 [==============================] - 168s 6s/ste

In [ ]:
# Evaluate the model
val_loss, val_acc = model.evaluate(X_val, y_val)
print(f'Validation Accuracy: {val_acc:.2f}')

7/7 [==============================] - 9s 1s/step - loss: 11.3647 - accuracy: 0.1244
Validation Accuracy: 0.12


In [ ]:
model.save('facial_recognition_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import pickle

# Save the label encoder
with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)


In [ ]:
# take a picture as input from user and predict based on model confidence threshold

def predict_image(model, image_path, threshold=0.9):

  image = cv2.imread(image_path)
  image = cv2.resize(image, (224, 224))
  image = image / 255.0

  predictions = model.predict(np.expand_dims(image, axis=0))

  predicted_index = np.argmax(predictions)

  predicted_person = label_encoder.classes_[predicted_index]

  # Check if the confidence is above the threshold
  if predictions[0][predicted_index] > threshold:
    return predicted_person
  else:
    return "Unknown"

# Take a picture as input from the user
image_path = input("Enter the path to the image: ")

# Predict the person in the image
predicted_person = predict_image(model, image_path)

# Print the predicted person
print(f"Predicted Person: {predicted_person}")


Enter the path to the image: /content/dataset/Hemel/Hemel.7.1445.26.jpg
1/1 [==============================] - 0s 95ms/step
Predicted Person: Unknown


In [ ]:
#Pre-trained Model for Checking accuracy

from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(10, activation='softmax')(x)

model2 = Model(inputs=base_model.input, outputs=predictions)

model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Callbacks for early stopping and learning rate reduction
early_stopping = EarlyStopping(patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(factor=0.5, patience=5)

history = model2.fit(datagen.flow(X_train, y_train, batch_size=32),
                    validation_data=(X_val, y_val),
                    epochs=12,
                    callbacks=[early_stopping, reduce_lr])

Epoch 1/12
26/26 [==============================] - 574s 22s/step - loss: 1.2450 - accuracy: 0.5236 - val_loss: 0.5266 - val_accuracy: 0.8905 - lr: 0.0010
Epoch 2/12
26/26 [==============================] - 569s 22s/step - loss: 1.1959 - accuracy: 0.5311 - val_loss: 0.4679 - val_accuracy: 0.9005 - lr: 0.0010
Epoch 3/12
26/26 [==============================] - 567s 22s/step - loss: 1.1629 - accuracy: 0.5485 - val_loss: 0.3989 - val_accuracy: 0.9801 - lr: 0.0010
Epoch 4/12
26/26 [==============================] - 568s 22s/step - loss: 1.0964 - accuracy: 0.5684 - val_loss: 0.3273 - val_accuracy: 0.9950 - lr: 0.0010
Epoch 5/12
26/26 [==============================] - 570s 22s/step - loss: 1.0880 - accuracy: 0.6007 - val_loss: 0.3003 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 6/12
26/26 [==============================] - 565s 23s/step - loss: 1.0815 - accuracy: 0.5858 - val_loss: 0.2691 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 7/12
26/26 [==============================] - 568s 22s/step - lo

In [ ]:
# Evaluate the model2
val_loss, val_acc = model2.evaluate(X_val, y_val)
print(f'Validation Accuracy: {val_acc:.2f}')

7/7 [==============================] - 105s 15s/step - loss: 0.1868 - accuracy: 1.0000
Validation Accuracy: 1.00


In [ ]:
model2.save('facial_recognition_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Because of the limitations of the available dataset a pre-trained model is better suited for our smart attendance system.